# DNBLab Jupyter Notebook zur SRU Abfrage

## SRU - Schnittstellenabfrage, Datenauslieferung und Ergebnisanzeige

Lassen Sie uns gemeinsamen folgenden Markdown *hier* einfügen:

Dieses DNBLab-Tutorial beschreibt eine Beispielabfrage über die SRU-Schnittstelle mit Python. In der Jupyter Notebook Umgebung kann der dokumentierte Code direkt ausgeführt und angepasst werden. Das Tutorial umfasst die exemplarische Anfrage und Ausgabe der Daten in MARC21-xml zur weiteren Verarbeitung. 

## Ganz wichtig, Zwischenspeichern nicht vergessen!

## Lassen Sie uns die Arbeitsumgebung einrichten! <a class="anchor" id="Teil1"></a>

In [10]:
import urllib.parse
from pyodide.http import open_url, pyfetch
from js import fetch
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree
import pandas as pd

## Lassen Sie uns die SRU-Abfrage starten ... <a class="anchor" id="Teil2"></a>

In [11]:
async def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query
         }
    
    r = await fetch(base_url + "?" + urllib.parse.urlencode(params))  
    r_text = await r.text()
    xml = soup(r_text, features="xml")
    records = xml.find_all('record', {'type':'Bibliographic'})
    
    
    if len(records) < 100:
        
        return records
    
    else:
        
        num_results = 100
        i = 101
        while num_results == 100:
            
            params.update({'startRecord': i})
            r = await fetch(base_url + "?" + urllib.parse.urlencode(params))  
            r_text = await r.text()
            xml = soup(r_text, features="xml")
            new_records = xml.find_all('record', {'type':'Bibliographic'})
            records+=new_records
            i+=100
            num_results = len(new_records)
            
        return records

## Lasses Sie uns ein MARC-Feld durchsuchen! <a class="anchor" id="Teil3"></a>

In [13]:
def parse_record(record):
    
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
    #idn
    idn = xml.xpath("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'
    
    # titel
    titel = xml.xpath("marc:datafield[@tag = '856']/marc:subfield[@code = 'u']", namespaces=ns)
    
    try:
        titel = titel[0].text
        #titel = unicodedata.normalize("NFC", titel)
    except:
        titel = "unkown"
        
    meta_dict = {"idn":idn,
                 "link":link}
    
    return meta_dict

# Lassen Sie uns gemeinsam alle Titel nach dem Suchwort 'Biene durchsuchen'!

In [14]:
records = await dnb_sru('tit=Hund')
print(len(records), 'Ergebnisse')

2069 Ergebnisse


## Lassen Sie uns das Ergebnis in einem Satz anzeigen...

In [8]:
print(len(records), 'Treffer wurden gefunden!')

2069 Treffer wurden gefunden!


## ... und nun die Treffer tabellisch darstellen.  <a class="anchor" id="Teil4"></a>

In [6]:
output = [parse_record(record) for record in records]
df = pd.DataFrame(output)
df

,idn,titel
0,1307583822,Das Lied der Biene
1,1304403807,Der Stich der Biene
2,1311597247,Die Biene Maja und ihre Abenteuer
3,1307587437,Die Biene Maja und ihre Abenteuer
4,130090240X,Die Biene Maja: Die schönsten Minuten-Geschi...
...,...,...
2064,1232876879,Postkarte
2065,1235784428,Briefkarte
2066,1238096719,Konvolut von Zeitungsausschnitten u. Ä. mit Be...
2067,1237361656,Konvolut von Programmen seines Theaterstücks B...


# Wie kann man das Ergebnis speichern? 

Mit der folgenden Funktion df.to_csv() werden die Ergebnisse als SRU_Titel.csv in das Jupyter-Verzeichnins der Einstiegsseite abgelegt und können dort heruntergeladen werden. 

In [9]:
df.to_csv("SRU_Titel.csv", index=False)